In [ ]:
import os,sys

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import date, datetime
import seaborn as sns
import sklearn
from sklearn.cross_validation import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
data_folder = '../../data/'

In [ ]:
df_airlines = pd.read_csv(os.path.join(data_folder,"airlines.csv"),sep=';')
df_airports = pd.read_csv(os.path.join(data_folder,"airports.csv"),sep=';')
df_flights = pd.read_csv(os.path.join(data_folder,"flights.csv"),sep=';')
df_planes = pd.read_csv(os.path.join(data_folder,"planes.csv"),sep=';')
df_weather = pd.read_csv(os.path.join(data_folder,"weather.csv"),sep=';')

## a) Perform a left join with flights and airlines by carrier field

In [ ]:
df_flights = df_flights.merge(df_airlines,on='carrier')
#df_flights

## b) Create a histogram plot of air_time field of flights dataframe 

In [ ]:
plt.figure(figsize=(10,10))
plt.title("histogram of air time")
plt.xlabel("air time in minutes")
plt.ylabel("number of flights")
histo = plt.hist(df_flights.air_time.dropna(),bins=100)
plt.savefig("fig/hist_air_time.png")

## c) Create a plot of number of flights per day.

In [ ]:
df_flights['dep_date'] = df_flights.apply(lambda r : date(r['year'],r['month'],r['day']),axis=1 )

In [ ]:
df_nb_flight_per_day = df_flights.reset_index().groupby('dep_date')['index'].count()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("number of flight per departure date")
plt.xlabel("departure date")
plt.ylabel("number of flights")
plt.plot(df_nb_flight_per_day)
plt.savefig('fig/nb_flight_per_day.png')

In [ ]:
df_flights['dep_day_of_week'] = df_flights['dep_date'].apply(lambda x : x.weekday())

## (d) What features would you use to forecast volume? 

I would use the volume in the recent past it self, just like for a time serie prediction (SARIMA), the day of week, and all weather. The problem here is that we clearly not under the time serie assumption as it is known that flights volume has a seasonnality over the year and here we have a sample with the size of a year.

## (e) Perform a logistic regression to model volume (do not worry on overfitting).

### Data preparation

In [ ]:
df_flights['ond'] = df_flights[['origin','dest']].apply(lambda x : x['origin']+x['dest'],axis=1)
df_flights['one'] = 1
df_nb_flight_per_day = df_nb_flight_per_day.reset_index()
df_nb_flight_per_day = df_nb_flight_per_day.rename(columns={'index':'nb_flight'})
df_weather['dep_date'] = df_weather.apply(lambda r : date(r['year'],r['month'],r['day']),axis=1 )

In [ ]:
df_nb_flights_orig = df_flights.groupby(['dep_date','origin'])['one'].count().reset_index().pivot_table(values='one',index='dep_date',columns='origin')

In [ ]:
df_nb_flights_orig['nb_flight_ground_truth'] = df_nb_flights_orig.apply(lambda x : x['EWR'] + x['JFK'] + x['LGA'],axis=1)

In [ ]:
df_weather=df_weather[[col for col in df_weather.columns if col not in ['year','month','day','hour']]]

In [ ]:
df_weather_orig_pivot = df_weather.pivot_table(values=[col for col in df_weather.columns if col not in ['dep_date','origin']],index='dep_date',columns='origin').reset_index()

In [ ]:
df_features = df_nb_flights_orig.reset_index().merge(df_weather_orig_pivot,on='dep_date')

In [ ]:
df_features['dep_day_of_week'] = df_features['dep_date'].apply(lambda x : x.weekday())

### Volume Modelisation with logistic regression

In [ ]:
logreg = LogisticRegression(C=1e4)

In [ ]:
X = df_features[[col for col in df_features.columns if col not in ['dep_date','EWR','JFK','LGA','nb_flight_ground_truth']]]
y = df_features['nb_flight_ground_truth']

In [ ]:
logreg.fit(X,y)

In [ ]:
df_features['predicted'] = logreg.predict(X)

In [ ]:
df_features['modelisation_error'] = df_features['predicted'] - y

In [ ]:
plt.figure(figsize=(20,20))
plt.title("flight volume modelisation using logistic regression")
plt.plot(df_features[['dep_date','predicted','nb_flight_ground_truth','modelisation_error']].set_index('dep_date'))

### f) How would you select features in that logistic regression to use this model to forecast number of flights for the following next days? (no code needed for this question)

I will select meaningfull features available in the future. for example we could use predicted weather information. I will also use day of week.

### g) (optional) Create a histogram plot of dep_delay field of flights dataframe.

In [ ]:
plt.figure(figsize=(10,10))
plt.title("departure delay histogram")
plt.xlabel("departure delay in minutes")
plt.ylabel("distribution of delays")
dep_delay_hist = plt.hist(df_flights['dep_delay'].dropna(),bins=100)
plt.savefig("fig/departure_delay_histogram.png")

### (h) Discretize dep_delay variable into dep_delay_cat, where values equal or below zero are a category itself, and the rest are categorized by deciles. Keep missing values as missing values.


In [ ]:
deciles = df_flights[df_flights.dep_delay > 0].dep_delay.quantile(q=list(map(lambda x : x/10,range(0,10)))).reset_index()

In [ ]:
deciles['index'] = deciles.index

In [ ]:
deciles = deciles.dep_delay.tolist()

In [ ]:
def discretization(x):
    if x <= 0 :
        return -1
    if x <= deciles[0] :
        return 0
    if x > deciles[-1]:
        return 9
    if x == 1 :
        return 0
    for i in range(1,9):
        if x > deciles[i] and x <= deciles[i+1]:
            return i
    if pd.isnull(x):
        return pd.np.NaN

In [ ]:
dep_delay_discretized = df_flights.dep_delay.apply(discretization)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("histogram of discretized departure delay")
plt.xlabel("delay category")
plt.ylabel("distribution of the categories")
disc_delay_hist = plt.hist(dep_delay_discretized.dropna())

In [ ]:
df_flights['dep_delay_discretized'] = df_flights.dep_delay.apply(discretization)

### (i) Create a classification model and perform the evaluation of performance.

#### Data preparation

In [ ]:
colmuns = list(filter(lambda x : x not in ['year','month','day','dep_delay',
                                          'sched_arr_time','flight','tailnum',
                                          'air_time','dest','hour','minute','time_hour',
                                          'dep_date','ond'], df_flights.columns))

In [ ]:
colmuns = list(filter(lambda x : x != 'origin', df_flights.columns))

In [ ]:
df_flights['time_hour'] = df_flights.apply(lambda r : datetime(r['year'],r['month'],r['day'],r['hour']),axis=1)

In [ ]:
df_weather['time_hour'] = df_weather.time_hour.apply(lambda x : datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

#### Dealing with missing values on weather using simple imputation by linear interpolation

We assume that data is missing under MCAR hypothesis

In [ ]:
weather_data_list = list()
for k,g in df_weather.groupby('origin'):
    g = g.set_index('time_hour').resample('H').interpolate('linear')
    g.origin = g.origin.fillna(value=k)
    weather_data_list.append(g)
df_weather_imputed = pd.concat(weather_data_list).reset_index()    

In [ ]:
if 'dep_date' in df_weather_imputed.columns :
    del df_weather_imputed['dep_date']

In [ ]:
df_flights_weather = df_flights.merge(df_weather_imputed,on=['time_hour','origin'],how='left')

From this we remove rows with missing values related to depature time

In [ ]:
"rows number before missing values drop {0}".format(df_flights_weather.shape[0])

In [ ]:
df_flights_weather = df_flights_weather.dropna().reset_index(drop=True)

In [ ]:
"rows number after missing values drop {0}".format(df_flights_weather.shape[0])

In [ ]:
weather_cols = [col for col in df_weather_imputed.columns if col not in ['time_hour','origin']]

In [ ]:
df_flights_cols = ['distance','dep_day_of_week']

In [ ]:
#X_all = df_flights_weather[weather_cols+df_flights_cols]
#y_all = df_flights_weather['dep_delay_discretized']

In [ ]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [ ]:
label_encode = LabelEncoder()
onehot_encode = OneHotEncoder()

In [ ]:
encoded_carrier_label = label_encode.fit_transform(df_flights_weather.carrier).reshape(*X_all.carrier.shape)

In [ ]:
encoded_carrier = onehot_encode.fit_transform(encoded_carrier_label.reshape(-1, 1)).toarray()

In [ ]:
carrier_mapping = dict(zip(list(range(16)),label_encode.inverse_transform(list(range(16)))))

In [ ]:
df_carrier_endoced = pd.DataFrame(encoded_carrier,columns=list(carrier_mapping.values()))

In [ ]:
df_flights_weather = df_flights_weather.join(df_carrier_endoced)

In [ ]:
X_all = df_flights_weather[weather_cols+df_flights_cols+list(carrier_mapping.values())]
X_all_no_carrier = df_flights_weather[weather_cols+df_flights_cols]
y_all = df_flights_weather['dep_delay_discretized']

#### Model building

##### We take care of imbalanced data set

In [ ]:
dep_delay_discretized_distrib = y_all.reset_index().groupby('dep_delay_discretized').count()

In [ ]:
dep_delay_discretized_distrib = 1/ dep_delay_discretized_distrib

In [ ]:
class_weights = dict(
    (dep_delay_discretized_distrib / sum(map(lambda x : x[1],dep_delay_discretized_distrib.reset_index().values))).reset_index().values
)

In [ ]:
decision_tree = DecisionTreeClassifier("entropy",random_state=0, class_weight=class_weights)

In [ ]:
original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 5}
gbt = GradientBoostingClassifier(**original_params)

In [ ]:
log_reg = LogisticRegression(C=1e5,class_weight=class_weights)

In [ ]:
scores = cross_val_score(decision_tree,X_all_no_carrier,y_all,cv=10)

In [ ]:
scores

In [ ]:
scores

In [ ]:
decision_tree.fit(X_all_no_carrier.iloc[100000:300000],y_all.iloc[100000:300000])

In [ ]:
set(decision_tree.predict(X_all_no_carrier.iloc[5:100000]))

In [ ]:
y_all.iloc[5:100]

In [ ]:
scores = cross_val_score(log_reg,X_all,y_all,cv=2)

In [ ]:
scores

In [ ]:
log_reg.fit(X_all.iloc[10000:30000],y_all.iloc[10000:30000])

In [ ]:
set(log_reg.predict(X_all.iloc[5:10000]))

In [ ]:
y_all.iloc[5:10000]

## (j) What is the performance you would predict in production for the model?